In [1]:
import numpy as np
import xarray as xr
import os
from tqdm.autonotebook import tqdm
import keras
from surgeNN import io,preprocessing
import datetime
import fnmatch
import gc
import tensorflow as tf
#NOTE: keras & tensorflow versions must align with the versions used to train the models (see req.txt)

/tmp/ipykernel_3770/501805711.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
2025-07-28 08:24:10.625577: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-28 08:24:10.698972: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Notebook to load trained model and apply to forcing:

In [2]:
#configure the script
performance_path = '/home/jovyan/surgeNN/results/nns_codec/performance/lstm/lstm_3h_den_helder-denhdr-nld-rws_mse_hp1_ndeg3_it0.nc'
model_fns = [os.path.join('/home/jovyan/surgeNN/results/nns_codec/keras_models/lstm',k) for k in os.listdir('/home/jovyan/test_surge_models/results/nns_codec/keras_models/lstm') if k.endswith('keras')]

predictor_path = 'gs://leap-persistent/timh37/HighResMIP/surgeNN_predictors/' #path to predictor data derived from highresmip model

#adapt script so that it can loop over multiple models: have base script work on single model, script around it to loop over performances?
out_path = '/home/jovyan/surgeNN/results/nns_hadgem3/codec_trained_predictions'
out_fn = 'predictions_hadgem3_forced_codec_trained_lstm'
predictor_vars = ['msl','u10','v10'] #must align with what was used training (to-do: store this in performance output atrributes)

In [3]:
performance = xr.open_dataset(performance_path)
hyperparams = performance.hyperparameters.copy(deep=True)
performance = performance.isel(i=0,drop=True) #model_i or model_it doesn't matter here, as o & n_steps is the same for all models
tg = performance.tg.values[0]

In [4]:
def load_and_predict(performance,tg,model_fns,predictor_path,predictor_vars,out_path,out_fn):
    performance = performance.sel(tg=tg)
    n_steps = int(performance.hyperparameters.sel(p='n_steps').isel(split=0).values)

    #load predictors
    predictors = io.Predictor(predictor_path)
    predictors.open_dataset(tg,predictor_vars,performance.n_cells)

    if 'intake_esm_attrs:source_id' in predictors.data.attrs and predictors.data.attrs['intake_esm_attrs:source_id'] == 'HadGEM3-GC31-HM':
        time = predictors.data.time
        where_diff_not_3h = np.where(np.diff(predictors.data.time) !=  datetime.timedelta(seconds=10800))[0][0] #01-01-2015 00:00:00 is missing, but also in predictand data -> not an issue?

        newtime = np.hstack([time.values,time.values[-1]])
        newtime[0:where_diff_not_3h+1] = time.values[0:where_diff_not_3h+1]
        newtime[where_diff_not_3h+2::] = time.values[where_diff_not_3h+1::]
        newtime[where_diff_not_3h+1] = newtime[where_diff_not_3h+1] - datetime.timedelta(seconds=10800)

        predictors.data = predictors.data.reindex(time=newtime,method='ffill') #repeat previous datapoint

    predictors.subtract_annual_means()
    predictors.deseasonalize()

    model_input = preprocessing.predictionInput(predictors)
    model_input.stack_predictor_coords()
    model_input.standardize()
    x_in = model_input.get_windowed_filtered_np_input(n_steps)

    for model_fn in model_fns:
        model = keras.models.load_model(model_fn)
        y_out = model.predict(x_in)

        #backtransform using era5 obs
        y_train_mean = np.nanmean(performance.o.sel(split='train'))
        y_train_sd = np.nanstd(performance.o.sel(split='train'),ddof=0)

        y_hat = y_out*y_train_sd + y_train_mean
        t_hat = predictors.data.time[n_steps-1::]

        out_ds = xr.Dataset(data_vars=dict(yhat=(["time","tg"], y_hat),y_train_mean = (["tg"],[y_train_mean]),y_train_sd = (["tg"],[y_train_sd])),
              coords=dict(time=t_hat,tg=[tg],lon=performance.lon,lat=performance.lat,),
              attrs=dict(model=model_fn, predictors=predictor_path,))

        out_ds['hyperparameters'] = hyperparams.isel(split=0,drop=True).isel(i=int(model_fn.split('_')[-2].replace('i','')))
        out_ds.attrs['model'] = model_fn
        out_ds.attrs['forcing'] = predictor_path
        out_ds.attrs['performance'] = performance_path
        out_ds.attrs['predictor_vars'] = predictor_vars

        if os.path.exists(out_path)==False:
            os.makedirs(out_path)
        out_ds.to_netcdf(os.path.join(out_path,out_fn+'_'+'_'.join(model_fn.split('_')[-2::]).replace('.keras','.nc')),mode='w')

        del model, out_ds
        tf.keras.backend.clear_session()
        gc.collect()


In [ ]:
out_ds = load_and_predict(performance,tg,model_fns,predictor_path,predictor_vars,out_path,out_fn)

/srv/conda/envs/notebook/lib/python3.10/site-packages/xarray/core/dataset.py:255: UserWarning: The specified Dask chunks separate the stored chunks along dimension "time" starting at index 198544. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.10/site-packages/xarray/core/dataset.py:255: UserWarning: The specified Dask chunks separate the stored chunks along dimension "lat_around_tg" starting at index 13. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.10/site-packages/xarray/core/dataset.py:255: UserWarning: The specified Dask chunks separate the stored chunks along dimension "lon_around_tg" starting at index 13. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
2025-07-28 08:25:34.228916: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load 

9090/9090 [==============================] - 17s 2ms/step
